## Package Import, Output Specification

In [10]:
### Todo ###
############
    # Histograms on sides
    # Two side-by-side plots
    # Good preset xrange,yrange for each axis option
    # Configurable marker size, opacity
    # clickable, selectable for transition metals?
    # Add more data -- which can be NaN -- to TOOLTIP 
    #   DFT bandgap, 
    
    # Selection on histogram (maybe write hist code in JS)
    # Configurable colors for predicted d states? rows on ptable?
    
import numpy as np
import pandas as pd
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt

from bokeh.plotting import *
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource, CDSView, CustomJS, Slider, Button, CheckboxButtonGroup,\
                         DataTable, TableColumn, NumberFormatter, Select
from bokeh.models.filters import Filter, GroupFilter
from bokeh import events
from bokeh.io import curdoc, show
from bokeh.models.tools import HoverTool
from bokeh.palettes import Category10
from bokeh.models.glyphs import Text

output_notebook()
output_file("icsd_tmc_data_explorer.html", title='ICSD TMC Data Explorer')

Loading BokehJS ...

## Defining Functions and Global Variables

In [84]:
### Set GLOBAL VARIABLES ##########################################################################
TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select"
FIG_TITLE = "ICSD Transition Metal Compounds Data Explorer"
SIZING_MODE = 'stretch_both'
STD_FONT_SIZE = 14
ANION_DICT = {'S':'Sulfides', 'Se':'Selenides', 'Cl':'Chlorides', 'O':'Oxides', 'F':'Fluorides', 
             'N':'Nitrides', 'P':'Phosphides'}
ANION_COLORS = Category10[len(ANION_DICT.keys())]
ANION_MARKERS = ['x', 'plus', 'triangle', 'circle', 'dash', 'asterisk', 'inverted_triangle']
COL_LENGTHEN = {
    'mm_dist':'Metal-Metal Distance (\u212B)', 'normed_dist':'Normalized M-M Distance',
    'delta':'M-M Distance - Alloy Bond Length', 'gii':'Global Instability Index',
    'sg_num':'Spacegroup #', 'tm_row':'Transition Metal Row', 'mn':'Mendeleev Number',
    'sg_sym':'Spacegroup','tm':'Transition Metal', 'ionic_r':'Ionic Radius (\u212B)', 
    'oxi':'Oxidation State', 'anions':'Anion', 'formula':'Chemical Formula',
    'd_state':'# of $d$ Electrons', 'cn':'Transition Metal Coord. #'}
COL_SHORTEN = {v:k for k,v in COL_LENGTHEN.items()}
INIT_X = 'mm_dist'
INIT_Y = 'gii'
###################################################################################################


def format_scatter_plot(scatter_plot):
    scatter_plot.title.align = 'center'
    scatter_plot.line([-999,999],[0,0], color='black')
    scatter_plot.line([0,0],[-999,999], color='black')
    ### Shaded area
    scatter_plot.patch([-999,-999,999,999],[-999,0.2,0.2,-999],alpha=0.2, line_width=0, 
            legend_label='Y \u2264 0.2')
    ### Legend
    scatter_plot.legend.background_fill_alpha = 0.8
    scatter_plot.legend.click_policy="hide"
    ### Fonts
    scatter_plot.xaxis.axis_label = COL_LENGTHEN[INIT_X]
    scatter_plot.yaxis.axis_label = COL_LENGTHEN[INIT_Y] 
    scatter_plot.xaxis.axis_label_text_font_style = "normal"
    scatter_plot.yaxis.axis_label_text_font_style = "normal"
    scatter_plot.title.text_font_size = f'{int(STD_FONT_SIZE*1.2)}pt'
    scatter_plot.xaxis.axis_label_text_font_size = f'{STD_FONT_SIZE}pt'
    scatter_plot.yaxis.axis_label_text_font_size = f'{STD_FONT_SIZE}pt'
    scatter_plot.xaxis.major_label_text_font_size = f'{int(STD_FONT_SIZE*0.9)}pt'
    scatter_plot.yaxis.major_label_text_font_size = f'{int(STD_FONT_SIZE*0.9)}pt'
    scatter_plot.legend.label_text_font_size = f'{STD_FONT_SIZE}pt'

## Data Import, Preparation, Filtering

In [76]:
### Import data
df = pd.read_csv('data/features_icsd_tmetal-compounds.csv')

### Filter out heteroanion, hetero-transition-metal compounds, Rh compounds (mistake in BVPs)
df = df[df.heteroanion == False]
df['heterotm'] = [tm1 != tm2 for tm1, tm2 in zip(df.tm1, df.tm2)]
df = df[df.heterotm == False]
    # The following are only relevant to hetero transition metal compounds, which we filter out
    # df['mean_cn'] = [cn1 + cn2 for cn1, cn2 in zip(df.cn1, df.cn2)]
    # df['mean_n'] = [n1 + n2 for n1, n2 in zip(df.n1, df.n2)]
    # df['mean_ionic_r'] = [ir1 + ir2 for ir1, ir2 in zip(df.ionic_r_1, df.ionic_r_2)]
df = df[df.anions != 'Sb'] #remove the SINGLE antimonide
df = df[df.tm1 != 'Rh'] # Filter Rh compounds (BVP param has mistake)
df = df[df.gii < 10] # Filter extremely high GII

### Create new columns
df['tm_row'] = [n + 1 for n in df.n1]
    
### Select only useful columns
df = df[['formula', 'sg_sym', 'sg_num',
         'anions', 'tm1', 'tm_row', 'cn1',
         'mm_dist', 'delta', 'normed_dist', 'ionic_r_1',
         'oxi1', 'd_state1', 'pred_d1', 
         'mn1',  'gii', 'n_elems']].dropna()
df = df.rename(columns={'tm1':'tm', 'cn1':'cn','ionic_r_1':'ionic_r','oxi1':'oxi',
                        'd_state1':'d_state', 'mn1':'mn'})

### Convert to CDS
source = ColumnDataSource(data=df)
# df.keys()

### Generate KDEs for all columns, anions
# kde_dict = {} # {col : [vals, kde]}
kde_dict = {} # {col : ['vals'vals, kde]}

def calc_kde(col_data):
    kernel = gaussian_kde(col_data)
    vals = np.linspace(min(col_data),max(col_data),1000)
    kde = kernel(vals).T
    return (vals, kde)

col_ls = []
vals_ls = []
kde_ls = []
anion_ls = []

for col in axis_options:
    for anion in ANION_DICT.keys():
        this_df = df[df.anions == anion]
        vals, kde = calc_kde(this_df[col])
        col_ls.append(col)
        vals_ls.append(vals)
        kde_ls.append(kde)
        anion_ls.append(anion)
        kde_dict.update({(col,'vals'):vals, (col,'kde'):kde, (col,'anion'):anion})
        
kde_df = pd.DataFrame.from_dict(kde_dict)
kde_source = ColumnDataSource(data=kde_df)

# # print(len(vals_ls))
# # print(len(kde_ls))
# # print(len(anion_ls))
# kde_df = pd.DataFrame(data=np.array([col_ls, vals_ls, kde_ls, anion_ls]).T, columns=['col','vals','kde','anion'])
# kde_df

gii                   mm_dist                 normed_dist            \
         vals       kde anion      vals       kde anion        vals       kde   
0    0.000288  0.685225     P  2.454779  0.287152     P    0.940642  0.655905   
1    0.003646  0.691095     P  2.458911  0.291655     P    0.942194  0.669648   
2    0.007003  0.696926     P  2.463043  0.296185     P    0.943746  0.683548   
3    0.010361  0.702714     P  2.467176  0.300741     P    0.945298  0.697603   
4    0.013719  0.708460     P  2.471308  0.305322     P    0.946850  0.711811   
..        ...       ...   ...       ...       ...   ...         ...       ...   
995  3.341112  0.015424     P  6.566471  0.020189     P    2.485042  0.042499   
996  3.344470  0.015263     P  6.570603  0.020042     P    2.486594  0.042129   
997  3.347827  0.015104     P  6.574735  0.019892     P    2.488146  0.041757   
998  3.351185  0.014946     P  6.578868  0.019741     P    2.489698  0.041382   
999  3.354543  0.014789     P  6.583000  0.019587     P    2.491250  0.041003   

              delta  ... ionic_r      sg_num                    tm_row  \
    anion      vals  ...   anion        vals       kde anion      vals   
0       P -0.165932  ...       P    8.000000  0.000729     P  4.000000   
1       P -0.161822  ...       P    8.217217  0.000732     P  4.002002   
2       P -0.157711  ...       P    8.434434  0.000734     P  4.004004   
3       P -0.153600  ...       P    8.651652  0.000737     P  4.006006   
4       P -0.149490  ...       P    8.868869  0.000740     P  4.008008   
..    ...       ...  ...     ...         ...       ...   ...       ...   
995     P  3.924109  ...       P  224.131131  0.001637     P  5.991992   
996     P  3.928220  ...       P  224.348348  0.001624     P  5.993994   
997     P  3.932330  ...       P  224.565566  0.001611     P  5.995996   
998     P  3.936441  ...       P  224.782783  0.001598     P  5.997998   
999     P  3.940552  ...       P  225.000000  0.001584     P  6.000000   

                            mn                  
          kde anion       vals       kde anion  
0    1.112513     P  19.000000  0.001429     P  
1    1.112469     P  19.057057  0.001433     P  
2    1.112337     P  19.114114  0.001437     P  
3    1.112117     P  19.171171  0.001441     P  
4    1.111807     P  19.228228  0.001444     P  
..        ...   ...        ...       ...   ...  
995  0.131422     P  75.771772  0.060592     P  
996  0.131457     P  75.828829  0.060372     P  
997  0.131482     P  75.885886  0.060141     P  
998  0.131497     P  75.942943  0.059898     P  
999  0.131501     P  76.000000  0.059644     P  

[1000 rows x 24 columns]

In [ ]:
kde_source

## Plotting

In [92]:
# Create default axes ranges for each plot option

### Initialize scatter plot
scatter_plot = figure(tools=TOOLS, x_range=(0, 9), y_range=(-0.01, 2), title=FIG_TITLE, 
                      sizing_mode=SIZING_MODE)
points_ls = []
for anion, anion_label, color, marker in zip(ANION_DICT.keys(), ANION_DICT.values(), ANION_COLORS, ANION_MARKERS):
    view = CDSView(source=source, filters=[GroupFilter(column_name='anions', group=anion)])
    points = scatter_plot.scatter(x='mm_dist', y='gii', source=source, view=view,
                                  fill_alpha=0.01, line_alpha=0.5, legend_label=anion_label,
                                  color=color, marker=marker, size=7)
    points_ls.append(points) #store for emitting changes in js_callback

### Initialize KDE plots
hkde_plot = figure(tools=TOOLS, height=200, sizing_mode='stretch_width', x_range=scatter_plot.x_range)
hkde_ls = []
for anion, anion_label, color, marker in zip(ANION_DICT.keys(), ANION_DICT.values(), ANION_COLORS, ANION_MARKERS):
    view = CDSView(source=kde_source, filters=[GroupFilter(column_name='anions', group=anion)])
    hkde_line = hkde_plot.scatter(x=f'{INIT_X}_vals', y=f'{INIT_X}_kde', source=kde_source, 
                                  view=view, fill_alpha=1, legend_label=anion_label,
                                  color=color, marker='circle', size=7)
    hkde_ls.append(hkde_line)
    
### Make callbacks
axis_code="""
        var col_name = cb_obj.value;
        var column = col_dict[col_name];
        //console.log(col_name);
        //console.log(column);
        points_ls.forEach(function(points) {{
            points.glyph.{var}.field = column;
        }});
        ax.axis_label = col_name
        source.change.emit();
    """

callbackx = CustomJS(args=dict(source=source, points_ls=points_ls, ax=scatter_plot.xaxis[0], 
                               col_dict=COL_SHORTEN), code=axis_code.format(var="x"))
callbacky = CustomJS(args=dict(source=source, points_ls=points_ls, ax=scatter_plot.yaxis[0], 
                               col_dict=COL_SHORTEN), code=axis_code.format(var="y"))

### Axis data selector tools
axis_options = ['gii','mm_dist','normed_dist','delta','ionic_r', 'sg_num','tm_row','mn'] 
pretty_axis_options = [COL_LENGTHEN[key] for key in axis_options]

xaxis_select = Select(title="X axis:", value=COL_LENGTHEN['mm_dist'], options=pretty_axis_options)
xaxis_select.js_on_change('value', callbackx)

yaxis_select = Select(title="Y axis:", value=COL_LENGTHEN['gii'], options=pretty_axis_options)
yaxis_select.js_on_change('value', callbacky)

### Tooltips
hover = HoverTool()
hover.tooltips ="""
    <div>
        <h3><center>@formula</center></h3>
        <div><strong>Spacegroup:    </strong>@sg_sym (@sg_num)</div>
        <div><strong>M-M Dist.:     </strong>@mm_dist \u212B</div>
        <div><strong>GII:           </strong>@gii</div>
        <div><strong>Clustered TM:  </strong>@tm</div>
        <div><strong>TM Coord. #:   </strong>@cn</div>
    </div>
"""
scatter_plot.add_tools(hover)

### Layout
format_scatter_plot(scatter_plot)
controls = Row(yaxis_select, xaxis_select)
layout = Column(hkde_plot, scatter_plot, controls, sizing_mode=SIZING_MODE)
show(layout)




# show(Column(hkde_plot))

In [23]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure
from bokeh.models.widgets import Slider, RangeSlider
from bokeh.models import ColumnDataSource, CustomJS

# generate random data
x = np.random.normal(10, 1, 1000)

# generate histogram
hist, edges = np.histogram(x, bins = 10)

# create dataframe of histogram
hist_df = pd.DataFrame({'count': hist, 'left':edges[:-1], 'right':edges[1:]})

# generate bokeh data source for initial histogram
bokeh_data = ColumnDataSource(hist_df)

# generate bokeh data source for new histogram calculation
x_df=pd.DataFrame(x,columns={'value'})

x_src=ColumnDataSource(x_df)


# generate bokeh plot
plot = figure()
plot.quad(source = bokeh_data, bottom = 0, top = 'count', left = 'left', right = 'right') 

callback = CustomJS(args=dict(source1=x_src,source2=bokeh_data), code="""
    var data = source1.data;
    var val = data['value'];
    var length = val.length;
    var size  = bin_size.value;
    var min = data_range.value[0];
    var max = data_range.value[1];

    // Decide number of bins needed
    var bins = Math.floor((max - min) / size); 

    // Put left edge point in an array
    var left_edge = new Array(bins);
    for (var i = 0; i < bins; i++){
        left_edge[i] = min+i*size;
    }

    // Put right edge point in an array
    var right_edge = new Array(bins);
    for (var i = 0; i < bins; i++){
        right_edge[i] = min+(i+1)*size;
    }

    // Initialize frequency
    var frequency = new Array(bins);
    for (var i = 0; i < bins; i++) frequency[i] = 0;

    // Calculate frequency for each bin
    for (var i = 0; i < length; i++) {
        if (val[i]==min) frequency[0]++;
        else if (val[i]==max) frequency[bins-1]++;
        else frequency[Math.floor((val[i] - min) / size)]++;

    }

    // Update data source with new bins and frequency
    var bokeh_data_new={};
    bokeh_data_new.count=frequency;
    bokeh_data_new.left=left_edge;
    bokeh_data_new.right=right_edge;

    source2.data=bokeh_data_new;
""")

# generate slider object to change bin widths interactively
binwidth_slider = Slider(start = 0,  end = 1, step = 0.02, value = 0.5 , js_callback=callback)
callback.args["bin_size"] = binwidth_slider

# generate range slider object to change min and max of the data points to be shown interactively
range_slider = RangeSlider(start = 0, end = 20,  step = 1, value = (0, 20), callback=callback)
callback.args["data_range"] = range_slider

widgets = WidgetBox(binwidth_slider, range_slider)

output_notebook()

show(row(plot, widgets))

AttributeError: unexpected attribute 'js_callback' to Slider, similar attributes are js_event_callbacks or js_property_callbacks

In [ ]:
def select_compounds():
    """Returns the data filtered by the global filters (derived from the widgets)"""
#     genre_val = genre.value
#     director_val = director.value.strip()
#     cast_val = cast.value.strip()
#     selected = df[
#         (movies.Reviews >= reviews.value) &
#         (movies.BoxOffice >= (boxoffice.value * 1e6)) &
#         (movies.Year >= min_year.value) &
#         (movies.Year <= max_year.value) &
#         (movies.Oscars >= oscars.value)
#     ]
#     if (genre_val != "All"):
#         selected = selected[selected.Genre.str.contains(genre_val)==True]
#     if (director_val != ""):
#         selected = selected[selected.Director.str.contains(director_val)==True]
#     if (cast_val != ""):
#         selected = selected[selected.Cast.str.contains(cast_val)==True]
    selected = df
    return selected


def update():
    """Updates the underlying data to reflect the filter and axes selections"""
    df = select_compounds()
    x_name = axis_map[x_axis.value]
    y_name = axis_map[y_axis.value]
    scatter_plot.xaxis.axis_label = x_axis.value
    scatter_plot.yaxis.axis_label = y_axis.value
    source.data = dict(
        x=df[x_name],
        y=df[y_name])
#         color=df["color"],
#         title=df["Title"])

def format_scatter_plot(scatter_plot):
    scatter_plot.title.align = 'center'
    scatter_plot.line([-999,999],[0,0], color='black')
    scatter_plot.line([0,0],[-999,999], color='black')
    ### Shaded area
    scatter_plot.patch([-999,-999,999,999],[-999,0.2,0.2,-999],alpha=0.2, line_width=0, 
            legend_label='GII \u2264 0.2', muted_alpha=0)
    ### Legend
    scatter_plot.legend.background_fill_alpha = 0.8
    scatter_plot.legend.click_policy="hide"
    ### Fonts
    scatter_plot.xaxis.axis_label_text_font_style = "normal"
    scatter_plot.yaxis.axis_label_text_font_style = "normal"
    scatter_plot.title.text_font_size = f'{int(STD_FONT_SIZE*1.2)}pt'
    scatter_plot.xaxis.axis_label_text_font_size = f'{STD_FONT_SIZE}pt'
    scatter_plot.yaxis.axis_label_text_font_size = f'{STD_FONT_SIZE}pt'
    scatter_plot.xaxis.major_label_text_font_size = f'{int(STD_FONT_SIZE*0.8)}pt'
    scatter_plot.yaxis.major_label_text_font_size = f'{int(STD_FONT_SIZE*0.8)}pt'
    scatter_plot.legend.label_text_font_size = f'{STD_FONT_SIZE}pt'
    
    
### Specify global presets
TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select"
CONTROLS = [x_axis, y_axis]
SIZING_MODE = 'stretch_both'
STD_FONT_SIZE = 16

### CustomJS commands
cb_change_axis = CustomJS(code=""" 
    var new_ax = cb_obj.value
    axis.
""")

x_axis.js_on_change('value', cb_change_x_axis)
y_axis.js_on_change('value', cb_change_y_axis)

### Creating configurable axes
axis_map = {'Global Instability Index': 'gii',
            'Metal-Metal Distance (\u212B)': 'mm_dist',
            'Normalized M-M Distance': 'normed_dist',
            'M-M Distance - Alloy Bond Length' : 'delta',
            'TM Coordination #' : 'cn',
            'TM Periodic Row #' :'tm_row',
            'Spacegroup #' : 'sg_num'
}
x_axis = Select(title="X Axis", options=sorted(axis_map.keys()), value="Metal-Metal Distance (\u212B)")
y_axis = Select(title="Y Axis", options=sorted(axis_map.keys()), value="Global Instability Index")

### Figure, title
scatter_plot = figure(tools=TOOLS, x_range=(0, 9), y_range=(-0.01, 2), 
           title="Metal-Metal Distance vs. GII", sizing_mode=SIZING_MODE)
scatter_plot.scatter('x', 'y', source=source, fill_alpha=0.01, line_alpha=0.5)#, 
#                      legend_label=label, color='color', marker=marker, size=9, line_width=1.5)


### Run
update()
format_scatter_plot(scatter_plot)
control_row = row(*CONTROLS)
p = column(scatter_plot, control_row)
show(p)

In [ ]:
### Specify presets
TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select"
SIZING_MODE = 'stretch_both'


### Figure, title
scatter_plot = figure(tools=TOOLS, width=1550, height=850, x_range=(0, 9), y_range=(-0.01, 2), 
           title="Metal-Metal Distance vs. GII", sizing_mode=SIZING_MODE)
scatter_plot.title.align = 'center'

### Points
for anion, label, color, marker in zip(anion_set, anion_names, colors, markers):
    view = CDSView(source=source, filters=[GroupFilter(column_name='anions', group=anion)])
    scatter_plot.scatter('mm_dist', 'gii', source=source, view=view, fill_alpha=0.01, line_alpha=0.5, 
                legend_label=label, color=color, marker=marker, size=9, muted_alpha=0, line_width=1.5)

### Axes lines, labels
scatter_plot.line([-999,999],[0,0], color='black')
scatter_plot.line([0,0],[-999,999], color='black')
scatter_plot.xaxis.axis_label = "Metal-Metal Distance (\u212B)"
scatter_plot.yaxis.axis_label = "Global Instability Index"
scatter_plot.xaxis.axis_label_text_font_style = "normal"
scatter_plot.yaxis.axis_label_text_font_style = "normal"

### Shaded area
scatter_plot.patch([-999,-999,999,999],[-999,0.2,0.2,-999],alpha=0.2, line_width=0, 
        legend_label='GII \u2264 0.2', muted_alpha=0)

### Legend
scatter_plot.legend.background_fill_alpha = 0.8
scatter_plot.legend.click_policy="hide"

### Fonts
STD_FONT_SIZE = 16
scatter_plot.title.text_font_size = f'{int(STD_FONT_SIZE*1.2)}pt'
scatter_plot.xaxis.axis_label_text_font_size = f'{STD_FONT_SIZE}pt'
scatter_plot.yaxis.axis_label_text_font_size = f'{STD_FONT_SIZE}pt'
scatter_plot.xaxis.major_label_text_font_size = f'{int(STD_FONT_SIZE*0.8)}pt'
scatter_plot.yaxis.major_label_text_font_size = f'{int(STD_FONT_SIZE*0.8)}pt'
scatter_plot.legend.label_text_font_size = f'{STD_FONT_SIZE}pt'

### Histograms on the edge
# create the horizontal histogram
# hhist, hedges = np.histogram(x, bins=20)
# hzeros = np.zeros(len(hedges)-1)
# hmax = max(hhist)*1.1

### Delta vs GII
# right = figure(tools=TOOLS, width=400, height=450, x_range=(0.4, 3), y_range=left.y_range, 
#            title="Normalized Metal-Metal Distance vs. GII")
# right.title.align = 'center'
# right.circle('x1', 'y', source=source, color='black', fill_alpha=0, line_alpha=0.2)
# right.xaxis.axis_label = "Metal-Metal Distance / Pure Metal "
# right.yaxis.visible = False
# right.xaxis.axis_label_text_font_style = "normal"
# # Shaded area
# right.patch([-9999,-9999,9999,9999],[-0.01,0.2,0.2,-0.01],alpha=0.2, line_width=0)

### Tooltips
hover = HoverTool()
hover.tooltips ="""
    <div>
        <h3><center>@formula</center></h3>
        <div><strong>Spacegroup:    </strong>@sg_sym</div>
        <div><strong>M-M Dist.:    </strong>@mm_dist \u212B</div>
        <div><strong>GII:    </strong>@gii</div>
        <div><strong>Atomic #:      </strong>@n1</div>
        <div><strong>Pred. d state: </strong>@pred_d1</div>
        <div><strong>Mendeleev #:   </strong>@mn1</div>
    </div>
"""
scatter_plot.add_tools(hover)


### SELECTORS
### Plotting anion selector
# anion_labels = list(set(df.anions))
# anion_buttons = CheckboxButtonGroup(labels=anion_labels)
# anion_buttons.js_on_click(CustomJS(code="""
#     console.log('anion_buttons: active=' + this.active, this.toString())
# """))

### Plotting transition metal selector
# tm_labels = list(set(df.tm1).union(set(df.tm2)))
# tm_labels = 
# tm_buttons = CheckboxButtonGroup(labels=tm_labels)
# tm_buttons.js_on_click(CustomJS(code="""
#     console.log('tm_buttons: active=' + this.active, this.toString())
# """))


### Creating layout, showing
p = gridplot([[scatter_plot]], sizing_mode=SIZING_MODE)#, right]])
# selectors = row(tm_buttons)
layout = column(p, sizing_mode=SIZING_MODE)#, selectors)
show(layout)

In [ ]:
import pandas as pd
from bokeh.io import curdoc
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource, DataRange1d, Select
from bokeh.plotting import figure, show
from bokeh.palettes import inferno

variables = ['sunlight_hours']#, 'Sunrise', 'Sunset']

def get_dataset(src, name, plottype):

    df = pd.DataFrame()
    df['date'] = pd.to_datetime(t)
    df['sun'] = src[name]

    return ColumnDataSource(data=df)

def make_plot(source, title, city):
    plot = figure(x_axis_type="datetime", plot_width=800, tools="", toolbar_location=None)
    plot.title.text = title
    plot.line('date','sun',line_width=2,line_color=clrs[1],legend=city,source=source)

# fixed attributes
plot.xaxis.axis_label = None
plot.yaxis.axis_label = "Sunlight [hours]"
plot.axis.axis_label_text_font_style = "bold"
#plot.x_range = DataRange1d(range_padding=0.0)
plot.grid.grid_line_alpha = 0.3

return plot

def update_plot(attrname, old, new):
    new_city = city_select.value
    plot.title.text = "Sunlight data for " + new_city

    src_update = get_dataset(sunlight, new_city, plottype_select.value)
    source.data.update(src_update.data)

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

i_city = 'Toronto'
plottype = 'sunlight_hours'

# make example data
yr = 2018
sites = pd.Series(['Resolute','Edmonton','Toronto'])
provs = pd.Series(['Nunavut','Alberta','Ontario'])
sunlight = pd.DataFrame()
sunlight['Toronto']  = pd.Series( [10,11,12,13,12,11,10] )
sunlight['Edmonton'] = pd.Series( [6,8,12,14,11,7,5] )
sunlight['Resolute'] = pd.Series( [4,6,10,16,11,5,2] )

t =  pd.date_range('1-1-' + str(yr),periods=7,freq='m')
N =  len(sites)

clrs = inferno(N)

cities = {}
for i in range(0,N):
    cities.update({sites[i]: {'city': sites[i], 'province': 
provs[i],'sun_hrs':sunlight[sites[i]],}})

city_select = Select(value=i_city, title='City', options=sorted(cities.keys()))
plottype_select = Select(value=plottype, title='Plot type', options=['Sunlight']) #, 'Sunrise', 'Sunset'])

source = get_dataset(sunlight, cities[i_city]['city'], plottype)

plot = make_plot(source, "Sunlight data for ",i_city)# + cities[city]['city'])

city_select.on_change('value', update_plot)
plottype_select.on_change('value', update_plot)

controls = column(city_select, plottype_select)

curdoc().add_root(row(plot, controls))
curdoc().title = "Sunlight"
